In [55]:
import pandas as pd
import numpy as np
from tkinter import filedialog
from tkinter.filedialog import askdirectory, askopenfilenames

In [51]:
#Variables
stress = 'THB'
SN = [x for x in range(1,31)]
Raw_0h = [x/100 for x in range(1,31)]
Raw_100h = [x+10/100 for x in range(1,31)]
Perc_Drift = [100*(x+0.21/x) for x in range(1,31)]
Real_Drift = [100*(x+0.21/x) for x in range(1,31)]
Important_Plots_Keywords = [
    "forcecurrent_1.20a",
    "vfc",
    "pfc",
    "uniformity",
    "wc",
    "ith",
    "div1e2"]

In [46]:
from openpyxl import Workbook
from openpyxl.styles import Alignment

def build_table(stress, hours, label, Raw_0h, Raw_100h, Perc_Drift, Real_Drift):
    # Header
    sheet = book.create_sheet(title=label+'_'+str(hours)+'H')
    sheet.merge_cells('A1:E1')
    cell_stress = sheet.cell(row=1, column=1)
    cell_stress.value = stress
    cell_stress.alignment = Alignment(horizontal='center', vertical='center')

    # SN Label
    sheet.merge_cells('A2:A3')
    SN_label = sheet.cell(row=2, column=1)
    SN_label.value = 'SN'
    SN_label.alignment = Alignment(horizontal='center', vertical='bottom')

    # Label Label
    sheet.merge_cells('B2:C2')
    cell_label = sheet.cell(row=2, column=2)
    cell_label.value = label
    cell_label.alignment = Alignment(horizontal='center', vertical='bottom')

    # Drift Label
    cell_label = sheet.cell(row=2, column=4)
    cell_label.value = 'Drift\n(%)'
    cell_label.alignment = Alignment(horizontal='center')


    # Real Drift Label
    cell_label = sheet.cell(row=2, column=5)
    cell_label.value = 'Real\nDrift (%)'
    cell_label.alignment = Alignment(horizontal='center')

    sheet['B3'] = '0h'
    sheet['B3'].alignment = Alignment(horizontal='center')
    sheet['C3'] = str(hours)+'h'
    sheet['C3'].alignment = Alignment(horizontal='center')

    # sht.range('A1').options(transpose=True).value = [1,2,3,4]

    # Dump SN
    for i in range(1,31):
        sheet['A{}'.format(i+3)] = i

    # Dump Raw 0H
    for count, value in enumerate(Raw_0h):
        sheet['B{}'.format(count+4)] = value    

    # Dump Raw 100H
    for count, value in enumerate(Raw_100h):
        sheet['C{}'.format(count+4)] = value

    # Dump Raw Drift
    for count, value in enumerate(Perc_Drift):
        sheet['D{}'.format(count+4)] = value

    # Dump Reak Drift
    for count, value in enumerate(Real_Drift):
        sheet['E{}'.format(count+4)] = value

In [50]:
def calculate_drift(Raw_Data,Ref_Data):
    real_drift = [abs((B-A)/B)*100 for A,B in zip(Raw_Data,Ref_Data)]
    return real_drift

In [53]:
def get_label_data(df, hour, label):
    Data = df.loc[(df[0] == hour)].filter(like=label)
    List_Data = Data.to_list()
    return List_Data

In [58]:
book = Workbook()
hour = 100
label = "forcecurrent_1.20a"
Raw_Data_DF = pd.read_csv(askopenfilenames(filetypes=(("CSV files", "*.csv"), ("All files", "*.*"))))
Ref_Data_DF = pd.read_csv(askopenfilenames(filetypes=(("CSV files", "*.csv"), ("All files", "*.*"))))

Raw_0h = get_label_data(Raw_Data_DF, 0, label)
Raw_100h = get_label_data(Raw_Data_DF, 100, label)

Ref_100h = get_label_data(Ref_Data_DF, 100, label)

Perc_Drift = calculate_drift(Raw_0h, Raw_100h)
Real_Drift = calculate_drift(Raw_100h, Ref_100h)

build_table(stress, hour, label, Raw_0h, Raw_100h, Perc_Drift, Real_Drift)
book.save('merging.xlsx')

FileNotFoundError: File b'' does not exist